In [1]:
# 0 := delete \n
# 1 := leave as it is

# model = gensim.models.Word2Vec.load('../../../pretrained_model/kor/ko.bin')

# from hangul_utils import split_syllables, join_jamos
# import gensim

In [2]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import data
import bpe
import utils
import pretrained_model as pm
import data_loader as dl
import trainer
import initializer as init
import tester
import model_util as mu
# import predictor

In [3]:
# df = data.getInitData()
# data.mk_initData(df)
# first_np, second_np, label_np = utils.process(df)

In [4]:
first_np, second_np, label_np  = data.getData()
first_np, second_np = utils.process_splitted(first_np, second_np)
first_ls, second_ls, ch2idx, max_len = utils.tokenize(first_np, second_np)
first2idx_np, second2idx_np = utils.encode(first_ls, second_ls, ch2idx, max_len)

linearized complete!
done tokenizing both data!
saved ch2idx to file!
encoding comlete!


In [5]:
pretrained_word2vec = pm.load_pretrained_model(ch2idx)
pretrained_word2vec = torch.tensor(pretrained_word2vec)

There are 27 / 133 pretrained vectors found.


In [6]:
train_first, test_first, train_second, test_second, train_labels, test_labels = train_test_split(
    first2idx_np, second2idx_np, label_np, test_size = 0.1, random_state = 43
)

train_first, val_first, train_second, val_second, train_labels, val_labels = train_test_split(
    train_first, train_second, train_labels, test_size = 0.1, random_state = 43
)

In [7]:
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(train_first,
                                                                   train_second,
                                                                   val_first,
                                                                   val_second,
                                                                   test_first,
                                                                   test_second,
                                                                   train_labels,
                                                                   val_labels,
                                                                   test_labels,
                                                                   batch_size=50)

In [8]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('ocr/tests')

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [9]:
# ====================
# set parameters here
# ====================

title = 'test8'
epochs = 40

vocab_size=len(ch2idx)
embed_dim = 200
hidden_size = 100
num_classes = 2
rnn_layers = 2
dropout = 0
learning_rate = 0.0001

model_name="RNN"
optim_name="Adam"
loss_fn_name="CEL"

pretrained_model=pretrained_word2vec
freeze_embedding=False,

In [10]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    dropout=dropout,
    learning_rate=learning_rate,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(model)

doing with pretrained model!!!
OCR(
  (emb): Embedding(133, 200)
  (lstm1): RNN(
    (rnn): LSTM(200, 100, num_layers=3, batch_first=True, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(200, 100, num_layers=3, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=400, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0, inplace=False)
  (dp2): Dropout(p=0, inplace=False)
)


In [11]:
trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.353228   | 85.580247  | 0.197958 | 92.89  | 110.53
   2    |   0.203694   | 92.362140  | 0.193038 | 93.15  | 110.55
   3    |   0.183011   | 93.094650  | 0.184187 | 93.52  | 111.05
   4    |   0.172904   | 93.526749  | 0.157986 | 94.37  | 110.53
   5    |   0.167464   | 93.658436  | 0.156329 | 94.59  | 110.29
   6    |   0.161556   | 93.983539  | 0.144652 | 94.93  | 110.31
   7    |   0.154053   | 94.160494  | 0.147679 | 94.63  | 110.56
   8    |   0.148616   | 94.427984  | 0.139565 | 95.11  | 110.39
   9    |   0.148013   | 94.510288  | 0.140064 | 95.37  | 110.59
  10    |   0.151010   | 94.477366  | 0.145625 | 94.70  | 110.50
  11    |   0.141698   | 94.818930  | 0.139856 | 95.04  | 111.29
  12    |   0.137964   | 94.888889  | 0.132243 | 95.22  | 110.54
  13    |   0.132628   | 95.074074  | 0.136343 | 95.5

KeyboardInterrupt: 

In [ ]:
tester.test(test_dataloader=test_dataloader,
            device=device, 
            model=model,
            title=title)

test loss:  0.17464827793107057
test acc:  95.53333333333333
saved precision and recall results to file!


(tensor([0., 1., 1.,  ..., 0., 0., 0.], device='cuda:0'),
 tensor([0., 1., 1.,  ..., 0., 0., 0.], device='cuda:0'))

In [ ]:
mu.graphModel(train_dataloader, model, writer)

uploaded model graph to tensorboard!


In [ ]:
mu.saveModel(title, model)